In [19]:
import pandas as pd
import math
from sklearn import model_selection
from sklearn import svm
import pyswarms as ps
import numpy as np
from sklearn.metrics import mean_squared_error

In [20]:
data = pd.read_csv('parkinsons.csv', delimiter=',')

In [65]:
x = data.drop(columns=['name', 'status']).to_numpy()
y = data['status'].to_numpy()

In [47]:
# four best features as per mid sems
x = data[['HNR','RPDE','DFA','PPE']]
x.shape

# set 1 as per correlation coeff -> 83.58% training, 78.46% testing
# x = data[['MDVP:Jitter(Abs)','MDVP:RAP','MDVP:PPQ','Jitter:DDP']]
# x.shape

(195, 4)

In [66]:
# 60% training set and 40% testing set
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size=0.4, random_state=0)

In [67]:
# SVM PARAMETER TUNING WITH C=2 
model_svm = svm.SVC(kernel='linear', C=0.87, gamma=0.61, degree=1000).fit(x_train, y_train)

In [68]:
model_svm.score(x_test, y_test)

0.9102564102564102

In [ ]:
# max_score = 0
# best_c = 1
# for x in range(1,100):
#     model = svm.SVC(C=x, kernel='linear', degree=3).fit(x_train, y_train)
#     score = model.score(x_test, y_test)
#     print(score)
#     if(score > max_score):
#         max_score = score
#         best_c = x

# print(max_score)
# print(best_c)

# 92.30769230769231% at c=2

In [72]:
def fitness_function(position):
    svmClassifier = svm.SVC(kernel='linear', gamma=position[0], C=position[1])
    svmClassifier.fit(x_train, y_train)
    y_train_pred = svmClassifier.predict(x_train)
    y_test_pred = svmClassifier.predict(x_test)
    mse_test = mean_squared_error(y_test, y_test_pred)
    rmse_test = math.sqrt(mse_test)
    print('Optimizing the Parameters ..... C = {c}, Gamma = {gamma}'.format(c=position[1], gamma=position[0]))
    mse_train = mean_squared_error(y_train, y_train_pred)
    rmse_train = math.sqrt(mse_train)
    # list(range(mse_f_train, rmse_f_train))
    retVal= [rmse_train, rmse_test] #Return a vector instead of single value. you can use the rmse_test and ignore rmse_train
    return retVal

In [73]:
def f(x):
    n_particles = x.shape[0]
    j = [fitness_function(x[i]) for i in range(n_particles)]
    return np.array(j)

In [74]:
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}
optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=2, options=options)
gamma_opt, C_opt = optimizer.optimize(f, iters=1000)

2019-12-10 22:26:05,306 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}




pyswarms.single.global_best:   0%|          |0/1000

Optimizing the Parameters ..... C = 0.6055649229846342, Gamma = 0.20722079199385512
Optimizing the Parameters ..... C = 0.8220484743684939, Gamma = 0.23209879859473836
Optimizing the Parameters ..... C = 0.34593487140260337, Gamma = 0.6696828497750937
Optimizing the Parameters ..... C = 0.679859111818788, Gamma = 0.816186818959687
Optimizing the Parameters ..... C = 0.9382621781299658, Gamma = 0.8599937851251234
Optimizing the Parameters ..... C = 0.3861414403785537, Gamma = 0.24073090240880823
Optimizing the Parameters ..... C = 0.9382807720334687, Gamma = 0.02808993107732094
Optimizing the Parameters ..... C = 0.18542544749211165, Gamma = 0.3600198994562105
Optimizing the Parameters ..... C = 0.8195372303649079, Gamma = 0.9368831528524014
Optimizing the Parameters ..... C = 0.15863306937590638, Gamma = 0.7473254690148801
Optimizing the Parameters ..... C = 0.9333456480897491, Gamma = 0.8981040223379068
Optimizing the Parameters ..... C = 0.8196730626973008, Gamma = 0.646183134990018


ValueError: operands could not be broadcast together with shapes (100,2) (100,) 

In [41]:
# PARAMETER TUNING USING GRID SEARCH CV
from sklearn.model_selection import GridSearchCV

In [40]:
model2 = svm.SVC(C=10, kernel='rbf', gamma="auto").fit(x_train, y_train)
model2.score(x_test, y_test)

0.8076923076923077

In [39]:
parameters = {'kernel':('rbf','linear'), 'C':range(3,100)}
svc = svm.SVC(gamma="auto")
clf = GridSearchCV(svc, parameters, cv=5)

In [63]:
#solo
parameters = {'C':range(1,1000)}
svc = svm.SVC(gamma="auto", kernel='linear')
clf = GridSearchCV(svc, parameters, cv=5)

In [ ]:
clf.fit(x_train, y_train)

In [58]:
sorted(clf.cv_results_.keys())

['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'param_C',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split1_test_score',
 'split2_test_score',
 'split3_test_score',
 'split4_test_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score']

In [59]:
y_pred = clf.predict(x_test)

In [60]:
x_test.shape

(78, 22)

In [61]:
clf.best_params_

{'C': 1}

In [62]:
clf.predict

<function sklearn.model_selection._search.BaseSearchCV.predict(self, X)>

In [42]:
# LOGISTIC REGRESSION
from sklearn.linear_model import LogisticRegression

In [5]:
x = data.drop(columns=['name', 'status'])
y = data['status']
data.columns

Index(['name', 'MDVP:Fo(Hz)', 'MDVP:Fhi(Hz)', 'MDVP:Flo(Hz)', 'MDVP:Jitter(%)',
       'MDVP:Jitter(Abs)', 'MDVP:RAP', 'MDVP:PPQ', 'Jitter:DDP',
       'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5',
       'MDVP:APQ', 'Shimmer:DDA', 'NHR', 'HNR', 'status', 'RPDE', 'DFA',
       'spread1', 'spread2', 'D2', 'PPE'],
      dtype='object')

In [230]:
# four best features as per mid sems
x = data[['HNR','RPDE','DFA','PPE']]
x.shape

# set 1 as per correlation coeff -> 83.58% training, 78.46% testing
x = data[['MDVP:PPQ','MDVP:Shimmer', 'MDVP:Shimmer(dB)',
          'Shimmer:APQ3', 'Shimmer:APQ5','MDVP:APQ','Shimmer:DDA']]
x.shape

(195, 7)

In [14]:
# 60% training set and 40% testing set
x_train, x_test, y_train, y_test = model_selection.train_test_split (x, y, test_size=0.33, random_state=0)

In [7]:
clf = LogisticRegression(random_state=0).fit(x_train, y_train)

/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [8]:
clf.score(x_test, y_test)

0.8923076923076924